In [3]:
!git clone https://github.com/AlexWan0/DomiKnowS.git
!cd DomiKnowS && git checkout notebook-program-name
!mv DomiKnowS/* .
!pip install -r requirements.txt

Cloning into 'DomiKnowS'...
remote: Enumerating objects: 15459, done.
remote: Total 15459 (delta 0), reused 0 (delta 0), pack-reused 15459
Receiving objects: 100% (15459/15459), 706.57 MiB | 30.65 MiB/s, done.
Resolving deltas: 100% (11162/11162), done.
Checking out files: 100% (314/314), done.
Branch 'notebook-program-name' set up to track remote branch 'notebook-program-name' from 'origin'.
Switched to a new branch 'notebook-program-name'
     |████████████████████████████████| 23.7 MB 3.8 kB/s 
     |████████████████████████████████| 11.1 MB 27.2 MB/s 
     |████████████████████████████████| 804.1 MB 2.7 kB/s 
     |████████████████████████████████| 2.3 MB 67.4 MB/s 
     |████████████████████████████████| 241 kB 56.7 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
  Created wheel for acls: filename=acls-1.0.2-py3-none-any.whl size=4828 sha256=a8181ee6265588a472b959c388c6aee0f40b8ff978e334f0ae2815b232e7f36f
  

In [4]:
!pip install torchtext==0.9 --no-dependencies

     |████████████████████████████████| 7.1 MB 7.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


Set logging to see training and testing results

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

First, build the graph that specifies the domain knowledge for this problem

In [6]:
from regr.graph import Graph, Concept, Relation
from regr.graph.relation import disjoint

Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
  review = Concept(name='review')

  positive = review(name='positive')
  negative = review(name='negative')

  disjoint(positive, negative)

Log file for dataNode is in: /content/logs/datanode.log


Define the readers for the labels and text


In [7]:
from regr.sensor.pytorch.sensors import ReaderSensor

review['text'] = ReaderSensor(keyword='text')

review[positive] = ReaderSensor(keyword='positive', label=True)
review[negative] = ReaderSensor(keyword='negative', label=True)

Model parameters

In [8]:
embed_size = 300
hidden_size = 100
num_classes = 2
drop_rate = 0.5

Define a sensor that converts the raw text to GloVe embedding vectors

In [9]:
from regr.sensor.pytorch.sensors import FunctionalSensor
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import torch

class EmbeddingSensor(FunctionalSensor):
  def __init__(self, *pres, **kwarg):
    super().__init__(*pres, **kwarg)

    self.vocab = GloVe(name='840B', dim=embed_size)
    self.tokenizer = get_tokenizer('spacy', language='en')
  
  def forward(self, *inputs):
    text = inputs[0]

    tokens_batch = [self.tokenizer(text)]

    emb_batch = []
    for tokens in tokens_batch:
      rev_emb = torch.empty((len(tokens), embed_size))
      for i, tok in enumerate(tokens):
        rev_emb[i] = self.vocab[tok]
      
      emb_batch.append(rev_emb)

    padded = pad_sequence(emb_batch)

    out = padded.to(device=self.device)

    return out

Connect the sensor to the graph such that it creates the embedding representation of the text based on the raw data.

In [10]:
review['text_embed'] = EmbeddingSensor('text')

INFO:torchtext.vocab:Downloading vectors from http://nlp.stanford.edu/data/glove.840B.300d.zip
.vector_cache/glove.840B.300d.zip: 2.18GB [06:50, 5.30MB/s]                            
INFO:torchtext.vocab:Extracting vectors into .vector_cache
INFO:torchtext.vocab:Loading vectors from .vector_cache/glove.840B.300d.txt
100%|█████████▉| 2195087/2196017 [04:20<00:00, 8569.02it/s]

Define a pytorch module for the LSTM model to produce RNN representations of the text

In [11]:
from torch import nn

class LSTMModule(nn.Module):
  def __init__(self):
    super(LSTMModule, self).__init__()

    self.rnn = nn.LSTM(embed_size, hidden_size, bidirectional=True)
    self.dropout = nn.Dropout(p=drop_rate)

  def forward(self, input):
    output, (h, c) = self.rnn(input)
    forward, backward = torch.chunk(output, 2, dim=2)
    comb = torch.cat((forward[-1,:,:], backward[0,:,:]), dim=1)

    return self.dropout(comb)

Define a learner using the previously specified pytorch module to create the RNN representation.

Then, specify learners that use a pytorch linear neural network to perform predictions based on that RNN representation.

In [12]:
from regr.sensor.pytorch.learners import ModuleLearner
from torch import nn

review['rnn_embed'] = ModuleLearner('text_embed', module=LSTMModule())

review[positive] = ModuleLearner('rnn_embed', module=nn.Linear(hidden_size * 2, num_classes))
review[negative] = ModuleLearner('rnn_embed', module=nn.Linear(hidden_size * 2, num_classes))

Define a learnable model from the previously specified graph.

In [13]:
from regr.program import SolverPOIProgram
from regr.program.metric import MacroAverageTracker, PRF1Tracker, DatanodeCMMetric
from regr.program.loss import NBCrossEntropyLoss

program = SolverPOIProgram(graph, inferTypes=['ILP', 'local/argmax'], loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})

Download IMDB data

In [14]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data/
!echo 'Extracting aclImdb_v1.tar.gz...'
!tar -xzf data/aclImdb_v1.tar.gz -C data/

--2021-07-24 02:31:00--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘data/aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  18.4MB/s    in 4.3s    

2021-07-24 02:31:04 (18.8 MB/s) - ‘data/aclImdb_v1.tar.gz.1’ saved [84125825/84125825]

Extracting aclImdb_v1.tar.gz...


Read data and place it in a list of dictionaries

In [18]:
import glob
import random
import os

def get_data(directory, label):
  data_all = []
  for path in glob.glob(os.path.join(directory, label + '/*.txt')):
    data_dict = {}
    with open(path, 'r') as f:
      data_dict['text'] = f.read()
      data_dict['positive'] = [1 if label == 'pos' else 0]
      data_dict['negative'] = [1 if label == 'neg' else 0]
    data_all.append(data_dict)
  return data_all

train_data = get_data('data/aclImdb/train', 'pos')
train_data.extend(get_data('data/aclImdb/train', 'neg'))
random.shuffle(train_data)

test_data = get_data('data/aclImdb/test', 'pos')
test_data.extend(get_data('data/aclImdb/test', 'neg'))
random.shuffle(test_data)

In [19]:
print(train_data[0])

{'text': 'Everyone should totally see this movie! It\'s freaking scary, but doesn\'t resort to lame "jump-out-at-you-just-to-surprise-you-and-pass-it-off-as-scary" things. It really is great. See this freaking awesome movie!!! The director is Stanley Kubrick, easily the greatest director who ever lived. Every single one of his movies are masterpieces, including this one. The Shining is about this family that goes to a hotel in the Colorado Rockies as caretakers for the winters, and get snowed in. Well, the house is haunted. The kid is psychic. The husband is easily impacted by evil haunted hotels, and...well...HILARITY ENSUES!!!! Not really. It becomes this gripping thriller where stuff gets thrown at the viewer from all different directions, and it gets scary. Not just the classic, "Here\'s Johnny" scene. It\'s memorable, but can\'t speak for the whole movies. It\'s one of those things where words don\'t explain it adequately, and you just gotta see it. So go on Netflix, and get it! G

Train/validation split

In [20]:
split_idx = int(len(train_data)*0.8)

Train the model

In [22]:
program.train(train_data[:5000],
              valid_set=train_data[split_idx:split_idx+2000],
              test_set=test_data[:2000], train_epoch_num=10, Optim=torch.optim.Adam, device='cuda')

INFO:regr.program.program:Epoch: 1
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.5942), 'negative': tensor(0.5936)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP


INFO:regr.program.program:{'positive': {'P': tensor(0.6802), 'R': tensor(0.6652), 'F1': tensor(0.6726)}, 'negative': {'P': tensor(0.6854), 'R': tensor(0.6660), 'F1': tensor(0.6756)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.6767), 'R': tensor(0.6756), 'F1': tensor(0.6762)}, 'negative': {'P': tensor(0.6828), 'R': tensor(0.6811), 'F1': tensor(0.6820)}}
INFO:regr.program.program:Validation:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.4091), 'negative': tensor(0.4099)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.7811), 'R': tensor(0.8892), 'F1': tensor(0.8316)}, 'negative': {'P': tensor(0.8685), 'R': tensor(0.7412), 'F1': tensor(0.7998)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.7812), 'R': tensor(0.8902), 'F1': tensor(0.8322)}, 'negative': {'P': tensor(0.8686), 'R': tensor(0.7422), 'F1': tensor(0.8004)}}
INFO:regr.program.program:Epoch: 2
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.3796), 'negative': tensor(0.3802)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8381), 'R': tensor(0.8354), 'F1': tensor(0.8368)}, 'negative': {'P': tensor(0.8403), 'R': tensor(0.8366), 'F1': tensor(0.8384)}}
INFO:regr.program.program: - - argmax


...

INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.6680), 'negative': tensor(0.6675)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8747), 'R': tensor(0.8427), 'F1': tensor(0.8584)}, 'negative': {'P': tensor(0.8457), 'R': tensor(0.8756), 'F1': tensor(0.8604)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8747), 'R': tensor(0.8427), 'F1': tensor(0.8584)}, 'negative': {'P': tensor(0.8457), 'R': tensor(0.8756), 'F1': tensor(0.8604)}}
INFO:regr.program.program:Epoch: 10
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.0500), 'negative': tensor(0.0500)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.9827), 'R': tensor(0.9823), 'F1': tensor(0.9825)}, 'negative': {'P': tensor(0.9825), 'R': tensor(0.9829), 'F1': tensor(0.9827)}}


INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.9827), 'R': tensor(0.9827), 'F1': tensor(0.9827)}, 'negative': {'P': tensor(0.9825), 'R': tensor(0.9829), 'F1': tensor(0.9827)}}
INFO:regr.program.program:Validation:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.6439), 'negative': tensor(0.6438)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8746), 'R': tensor(0.8556), 'F1': tensor(0.8650)}, 'negative': {'P': tensor(0.8556), 'R': tensor(0.8746), 'F1': tensor(0.8650)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8746), 'R': tensor(0.8556), 'F1': tensor(0.8650)}, 'negative': {'P': tensor(0.8556), 'R': tensor(0.8746), 'F1': tensor(0.8650)}}
INFO:regr.program.program:Testing:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.6747), 'negative': tensor(0.6749)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8590), 'R': tensor(0.8393), 'F1': tensor(0.8491)}, 'negative': {'P': tensor(0.8431), 'R': tensor(0.8617), 'F1': tensor(0.8523)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8590), 'R': tensor(0.8393), 'F1': tensor(0.8491)}, 'negative': {'P': tensor(0.8431), 'R': tensor(0.8617), 'F1': tensor(0.8523)}}
